In [160]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [161]:
# import os

# os.environ['PYSPARK_PYTHON'] = "./environment/bin/python"
# spark = SparkSession.builder.config(
#     "spark.archives",  # 'spark.yarn.dist.archives' in YARN.
#     "pyspark_conda_env.tar.gz#environment").getOrCreate()

# Names

In [162]:
# import os

# os.environ['PYSPARK_DRIVER_PYTHON'] = 'python'

Names.csv 
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dodaj kolumnę w której wyliczysz wzrost w stopach (feet)
* Odpowiedz na pytanie jakie jest najpopularniesze imię?
* Dodaj kolumnę i policz wiek aktorów 
* Usuń kolumny (bio, death_details)
* Zmień nazwy kolumn - dodaj kapitalizaję i usuń _
* Posortuj dataframe po imieniu rosnąco

In [163]:
# import time

# start = time.time()
# filepath = "dbfs:/FileStore/tables/Files/names.csv"
filepath = "names.csv"
names_df = spark.read.format("csv") \
          .option("header","true") \
          .option("inferSchema","true") \
          .load(filepath)
# end = time.time()

# display(names_df)

**Add current timestamp**

In [164]:
from pyspark.sql.functions import *

# names_df = names_df.withColumn("execution_time", lit(str(end - start)))
names_df = names_df.withColumn("current_timestamp", current_timestamp())

display(names_df.select("current_timestamp"))

DataFrame[current_timestamp: timestamp]

**Add height column in feet**

In [165]:
names_df = names_df.withColumn("height (feet)", col("height") / 2.54 / 12)

#  display(names_df.select("height (feet)"))

**Most popular name**

In [166]:
names_df.groupBy("name") \
    .agg(count("*")) \
    .sort(desc("count(1)")) \
    .show()

+-------------------+--------+
|               name|count(1)|
+-------------------+--------+
|     David Campbell|       4|
|      David Bradley|       4|
|     James Morrison|       3|
|      Robert Gordon|       3|
|       Frank Miller|       3|
|      Michael Jones|       3|
|   Michael Williams|       3|
|       Charles Lane|       3|
|         David Rose|       3|
|      Paul Williams|       3|
|Aleksandr Mikhaylov|       3|
|         John Moore|       3|
|     David Mitchell|       3|
|       Justin Kelly|       3|
|        Ian Roberts|       3|
|        Tony Martin|       3|
|   Charles Williams|       3|
|       James Murray|       3|
|          John Wood|       3|
|   William Marshall|       3|
+-------------------+--------+
only showing top 20 rows



**Add column with actors age**

Get birth timestamp

In [167]:
# date_of_birth

names_df = names_df.withColumn("birth", regexp_replace("date_of_birth","\.", "-")) \
               .withColumn("birth", regexp_replace("birth", "[A-Za-z,]", "")) \
               .withColumn("birth_timestamp", to_timestamp(col("birth"))) 
               # .select(col("birth"), when(to_timestamp(col("birth"),"yyyy-MM-dd").isNotNull, to_timestamp(col("birth"),"yyyy-MM-dd")))
# display(names_df.select("birth_timestamp"))

Get death timestamp

In [168]:
# date_of_death
names_df = names_df.withColumn("death", regexp_replace("date_of_death","\.", "-")) \
               .withColumn("death", regexp_replace("death", "[A-Za-z,]", "")) \
               .withColumn("death_timestamp", to_timestamp(col("death"), "dd-mm-yyyy"))
               # .select(col("birth"), when(to_timestamp(col("birth"),"yyyy-MM-dd").isNotNull, to_timestamp(col("birth"),"yyyy-MM-dd")))
# display(names_df.select("death_timestamp"))

Calculate age

In [169]:
names_df = names_df.withColumn("Age", months_between(col("death_timestamp"), col("birth_timestamp")) / 12)
# names_df = names_df.withColumn("Age", months_between(current_date(), col("birth_timestamp")) / 12)

# display(names_df.select(col("Age")))

**Drop bio and death_details**

In [170]:
names_df = names_df.drop("bio", "death_details")

# display(names_df)

**Normalize column names**

In [171]:
columns = names_df.columns

Capitalize

In [172]:
columns = list(map(str.capitalize, columns))

Remove underscores

In [173]:
columns = list(map(lambda x: x.replace("_", " "), columns))

Set new columns

In [174]:
names_df = names_df.toDF(*columns)

# display(names_df)

**Sort DataFrame**

In [175]:
names_df = names_df.sort("Name")

# display(names_df)

# Movies

Movies.csv
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dodaj kolumnę która wylicza ile lat upłynęło od publikacji filmu
* Dodaj kolumnę która pokaże budżet filmu jako wartość numeryczną, (trzeba usunac znaki walut)
* Usuń wiersze z dataframe gdzie wartości są null

In [176]:
# filepath = "dbfs:/FileStore/tables/Files/movies.csv"
filepath = "movies.csv"
movies_df = spark.read.format("csv") \
          .option("header","true") \
          .option("inferSchema","true") \
          .load(filepath)

display(movies_df)

DataFrame[imdb_title_id: string, title: string, original_title: string, year: string, date_published: string, genre: string, duration: int, country: string, language: string, director: string, writer: string, production_company: string, actors: string, description: string, avg_vote: string, votes: int, budget: string, usa_gross_income: string, worlwide_gross_income: string, metascore: double, reviews_from_users: double, reviews_from_critics: double]

**Current timestamp**

In [177]:
movies_df = movies_df.withColumn("current_timestamp", current_timestamp())

# display(movies_df.select("current_timestamp"))

**Year differnce**

In [178]:
movies_df = movies_df.withColumn("year_diff", months_between(col("current_timestamp"), to_timestamp(col("year"), "yyyy")) / 12)

# display(movies_df.select("year_diff"))

**Numeric budget**

In [179]:
from pyspark.sql.types import IntegerType

movies_df = movies_df.withColumn("budget_num", regexp_replace("budget", "[^0-9.]", "").cast(IntegerType()))

**Drop missing values**

In [180]:
movies_df = movies_df.dropna()

# display(movies_df)

# Ratings

ratings.csv
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dla każdego z poniższych wyliczeń nie bierz pod uwagę `nulls` 
* Dodaj nowe kolumny i policz mean i median dla wartości głosów (1 d 10)
* Dla każdej wartości mean i median policz jaka jest różnica między weighted_average_vote
* Kto daje lepsze oceny chłopcy czy dziewczyny dla całego setu
* Dla jednej z kolumn zmień typ danych do `long`

In [181]:
# filepath = "dbfs:/FileStore/tables/Files/ratings.csv"
filepath = "ratings.csv"
ratings_df = spark.read.format("csv") \
              .option("header","true") \
              .option("inferSchema","true") \
              .load(filepath)

display(ratings_df)

DataFrame[imdb_title_id: string, weighted_average_vote: double, total_votes: int, mean_vote: double, median_vote: double, votes_10: int, votes_9: int, votes_8: int, votes_7: int, votes_6: int, votes_5: int, votes_4: int, votes_3: int, votes_2: int, votes_1: int, allgenders_0age_avg_vote: double, allgenders_0age_votes: double, allgenders_18age_avg_vote: double, allgenders_18age_votes: double, allgenders_30age_avg_vote: double, allgenders_30age_votes: double, allgenders_45age_avg_vote: double, allgenders_45age_votes: double, males_allages_avg_vote: double, males_allages_votes: double, males_0age_avg_vote: double, males_0age_votes: double, males_18age_avg_vote: double, males_18age_votes: double, males_30age_avg_vote: double, males_30age_votes: double, males_45age_avg_vote: double, males_45age_votes: double, females_allages_avg_vote: double, females_allages_votes: double, females_0age_avg_vote: double, females_0age_votes: double, females_18age_avg_vote: double, females_18age_votes: double,

**Drop missing values**

In [182]:
ratings_df = ratings_df.dropna()

**Current timestamp**

In [183]:
ratings_df = ratings_df.withColumn("current_timestamp", current_timestamp())

# display(ratings_df.select("current_timestamp"))

**Mean votes column**

In [184]:
ratings_df = ratings_df.withColumn("mean_votes", (lit(10) * col("votes_10") + lit(9) * col("votes_9") \
                                                 + lit(8) * col("votes_8") + lit(7) * col("votes_7") \
                                                 + lit(6) * col("votes_6") + lit(5) * col("votes_5") \
                                                 + lit(4) * col("votes_4") + lit(3) * col("votes_3") \
                                                  + lit(2) * col("votes_2") + col("votes_1")) / \
                                                 (col("votes_10") + col("votes_9") + col("votes_8") \
                                                 + col("votes_7") + col("votes_6") + col("votes_5") \
                                                 + col("votes_4") + col("votes_3") + col("votes_2") \
                                                 + col("votes_1")))

In [185]:
ratings_df.select("votes_10", "votes_9", "votes_8", "votes_7", "votes_6", "votes_5", "votes_4", "votes_3", "votes_2", "votes_1", "mean_votes").show()

+--------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------------------+
|votes_10|votes_9|votes_8|votes_7|votes_6|votes_5|votes_4|votes_3|votes_2|votes_1|        mean_votes|
+--------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------------------+
|      12|      4|     10|     43|     28|     28|      9|      1|      5|     14| 5.896103896103896|
|     210|    225|    436|    641|    344|    169|     66|     39|     20|     87| 6.936522127849799|
|     285|    301|    591|    727|    443|    199|     85|     27|     18|    397| 6.470224536283761|
|    3661|   1741|   3314|   3963|   2876|   1928|    978|    701|    577|   2474| 6.440867960203485|
|       4|      3|      8|     20|     52|     27|      8|      5|      2|      1| 5.923076923076923|
|     620|    439|    763|    872|    484|    232|    140|     40|     31|    545| 6.651464234277484|
|    3477|   2230|   3214|   2249|   1179|    605|    340|    181|    133|    267|

**Median votes column**

In [186]:
import numpy as np

def median(*args):
    middle = int(np.sum(list(args)) / 2)
    sum_ = 0
    for median, count in list(enumerate(args)):
        sum_ += count
        if sum_ >= middle:
            break
    return 10 - median
    # return int(numpy.median([(value + 1) for value, count_ in reversed(list(enumerate(args))) for vote in range(count_)]))
    # return int(numpy.median([int(x) for x in args]))

# User defined function
udf_median = udf(median, IntegerType())

In [187]:
ratings_df = ratings_df.withColumn("median_votes", udf_median(ratings_df["votes_10"],
                                                              ratings_df["votes_9"],
                                                              ratings_df["votes_8"],
                                                              ratings_df["votes_7"],
                                                              ratings_df["votes_6"],
                                                              ratings_df["votes_5"],
                                                              ratings_df["votes_4"],
                                                              ratings_df["votes_3"],
                                                              ratings_df["votes_2"],
                                                              ratings_df["votes_1"]))

In [188]:
ratings_df.select("votes_10", "votes_9", "votes_8", "votes_7", "votes_6", "votes_5", "votes_4", "votes_3", "votes_2", "votes_1", "median_votes").show()

+--------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------------+
|votes_10|votes_9|votes_8|votes_7|votes_6|votes_5|votes_4|votes_3|votes_2|votes_1|median_votes|
+--------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------------+
|      12|      4|     10|     43|     28|     28|      9|      1|      5|     14|           6|
|     210|    225|    436|    641|    344|    169|     66|     39|     20|     87|           7|
|     285|    301|    591|    727|    443|    199|     85|     27|     18|    397|           7|
|    3661|   1741|   3314|   3963|   2876|   1928|    978|    701|    577|   2474|           7|
|       4|      3|      8|     20|     52|     27|      8|      5|      2|      1|           6|
|     620|    439|    763|    872|    484|    232|    140|     40|     31|    545|           7|
|    3477|   2230|   3214|   2249|   1179|    605|    340|    181|    133|    267|           8|
|     552|    564|   1303|   1742|    72

**Weighted average score and mean difference**

In [189]:
ratings_df = ratings_df.withColumn("avg_mean_diff", col("weighted_average_vote") - col("mean_votes"))

In [191]:
ratings_df.select("avg_mean_diff").show()

+--------------------+
|       avg_mean_diff|
+--------------------+
|0.003896103896104286|
| 0.06347787215020073|
|  0.6297754637162383|
| -0.1408679602034848|
|-0.02307692307692...|
|  0.6485357657225155|
| -0.0436036036036036|
|  0.5520780322307042|
| 0.16942321181273723|
|-0.06269724384599229|
| 0.13705680588418723|
| 0.18315098468271263|
| 0.30883956386292866|
|-0.08854166666666696|
|0.031493881320711736|
| -0.0579671784012703|
|-0.07096092925026376|
| 0.23827424529077845|
| 0.36584022038567454|
|0.006319045514803001|
+--------------------+
only showing top 20 rows



**Weighted average score and median difference**

In [193]:
ratings_df = ratings_df.withColumn("avg_median_diff", col("weighted_average_vote") - col("median_votes"))

In [194]:
ratings_df.select("avg_median_diff").show()

+--------------------+
|     avg_median_diff|
+--------------------+
|-0.09999999999999964|
|                 0.0|
| 0.09999999999999964|
| -0.7000000000000002|
|-0.09999999999999964|
|  0.2999999999999998|
|-0.20000000000000018|
|  0.2999999999999998|
| 0.09999999999999964|
|                 0.0|
|  0.3000000000000007|
| 0.09999999999999964|
| -0.2999999999999998|
|                 0.0|
|-0.09999999999999964|
|                -0.5|
|                 0.0|
| 0.09999999999999964|
| 0.09999999999999964|
| 0.09999999999999964|
+--------------------+
only showing top 20 rows



On average women vote higher

In [195]:
ratings_df.select(mean("males_allages_avg_vote"), mean("females_allages_avg_vote")).show()

+---------------------------+-----------------------------+
|avg(males_allages_avg_vote)|avg(females_allages_avg_vote)|
+---------------------------+-----------------------------+
|           6.17564751589357|            6.371356251471579|
+---------------------------+-----------------------------+



**Cast to long**

In [196]:
from pyspark.sql.types import LongType

ratings_df.select(col("total_votes").cast(LongType())).show()

+-----------+
|total_votes|
+-----------+
|        154|
|       2237|
|       3073|
|      22213|
|        130|
|       4166|
|      13875|
|       5895|
|      55601|
|       4753|
|     109038|
|      10054|
|       5136|
|       3072|
|      86620|
|       1889|
|       4735|
|      12157|
|       5082|
|       2263|
+-----------+
only showing top 20 rows



In [197]:
spark.stop()